In [ ]:
# Celda 1 — Configuración e imports

from pathlib import Path
import numpy as np
import pandas as pd
import librosa

# Carpeta con los .wav (una subcarpeta por especie)
IN_DIR = "../data/Tinamidae"

# Archivo de salida
OUT_CSV = "../traits/output_csv/SC.csv"

# Audio
SR_TARGET = 44100  # Hz

# STFT (ventana Hann de 1024 muestras, 50% solape, escala lineal)
WIN_LENGTH  = 1024
HOP_LENGTH  = WIN_LENGTH // 2
STFT_CENTER = False

# Filtrado por energía (silencios)
USE_ENERGY_THRESH = True
ENERGY_DB_FLOOR   = -60.0   # dB relativos al máximo RMS del archivo

# Banda de interés (BPF = 100–10000 Hz)
FMIN_BAND = 100.0   # Hz
FMAX_BAND = 10000.0 # Hz


In [ ]:
# Celda 2 — Utilidades: I/O, z-score, STFT, banda, RMS y SC por frame

def list_wavs(base_dir: str):
    base = Path(base_dir)
    return sorted([p for p in base.rglob("*.wav") if p.is_file()])

def species_from_path(p: Path, base_dir: str):
    """
    Asume estructura: base_dir/especie/archivo.wav
    Devuelve el nombre de la carpeta inmediatamente bajo base_dir.
    """
    base = Path(base_dir).resolve()
    rel  = p.resolve().relative_to(base)
    return rel.parts[0] if len(rel.parts) > 1 else "UNKNOWN"

def load_mono(fp: Path):
    """
    Carga audio mono a SR_TARGET.
    """
    y, sr = librosa.load(str(fp), sr=SR_TARGET, mono=True)
    return y.astype(np.float64, copy=False), SR_TARGET

def zscore_signal(y: np.ndarray, eps: float = 1e-12):
    """
    Señal estandarizada: (y - media) / sigma.
    Lo usamos para mantener consistencia con A, SF y H.
    """
    if y.size == 0:
        return y
    mu = float(np.mean(y))
    sd = float(np.std(y, ddof=0))
    return (y - mu) / (sd + eps)

def stft_mag(y_hat: np.ndarray):
    """
    Espectrograma de MAGNITUD |STFT| (frecuencia x tiempo),
    usando ventana Hann y escala lineal.
    """
    D = librosa.stft(
        y=y_hat,
        n_fft=WIN_LENGTH,
        hop_length=HOP_LENGTH,
        center=STFT_CENTER,
        window="hann",
    )
    S = np.abs(D)
    freqs = librosa.fft_frequencies(sr=SR_TARGET, n_fft=WIN_LENGTH)
    return S, freqs

def apply_band_mask(S: np.ndarray, freqs: np.ndarray,
                    fmin: float = FMIN_BAND, fmax: float = FMAX_BAND):
    """
    Recorta S a [fmin, fmax] sin cambiar el número de frames.
    """
    mask = (freqs >= fmin) & (freqs <= fmax)
    if not np.any(mask):
        return S, freqs
    return S[mask, :], freqs[mask]

def frame_rms_db(y_hat: np.ndarray):
    """
    RMS por frame en dB, relativo al máximo RMS del archivo.
    Sirve para filtrar frames silenciosos (< -60 dB).
    """
    rms = librosa.feature.rms(
        y=y_hat,
        frame_length=WIN_LENGTH,
        hop_length=HOP_LENGTH,
        center=STFT_CENTER,
    )[0]
    rms_db = librosa.amplitude_to_db(rms, ref=np.max)  # ≤ 0 dB
    return rms_db

def spectral_centroid_frames(S: np.ndarray, freqs: np.ndarray):
    """
    Centroide espectral por frame (Hz) usando magnitud |S| como peso:

        SC_t = sum_f f * S[f,t] / sum_f S[f,t]

    Devuelve SC_hz (vector 1D de longitud n_frames).
    """
    num = (S * freqs[:, None]).sum(axis=0)
    den = S.sum(axis=0)
    sc_hz = np.divide(
        num,
        den,
        out=np.full_like(num, np.nan, dtype=float),
        where=den > 0,
    )
    return sc_hz


In [ ]:
# Celda 3 — SC por archivo: \hat{SC} = mediana(SC_t en frames válidos)

files = list_wavs(IN_DIR)
print(f"Archivos .wav encontrados: {len(files)}")

rows = []

for fp in files:
    especie = species_from_path(fp, IN_DIR)
    try:
        # Carga y estandariza señal
        y, sr   = load_mono(fp)
        y_hat   = zscore_signal(y)

        # Espectrograma de magnitud y banda 100–10k
        S, freqs = stft_mag(y_hat)
        S, freqs = apply_band_mask(S, freqs, FMIN_BAND, FMAX_BAND)

        n_frames_total = int(S.shape[1])

        # Filtrado por energía (silencios)
        valid_mask = np.ones(n_frames_total, dtype=bool)
        if USE_ENERGY_THRESH:
            rms_db = frame_rms_db(y_hat)
            nT = min(n_frames_total, rms_db.shape[0])
            valid_mask[:] = False
            valid_mask[:nT] = rms_db[:nT] >= ENERGY_DB_FLOOR

        n_frames_valid = int(np.sum(valid_mask))

        if n_frames_valid == 0:
            rows.append({
                "species": especie,
                "relpath": str(fp),
                "sr": sr,
                "T_frames_total": n_frames_total,
                "T_frames_valid": 0,
                "SC_hat_file_hz": np.nan,
            })
            continue

        # SC_t en Hz para frames válidos y mediana por archivo
        sc_hz = spectral_centroid_frames(S[:, valid_mask], freqs)
        SC_hat_file = float(np.nanmedian(sc_hz))

        rows.append({
            "species": especie,
            "relpath": str(fp),
            "sr": sr,
            "T_frames_total": n_frames_total,
            "T_frames_valid": n_frames_valid,
            "SC_hat_file_hz": SC_hat_file,  # \hat{SC} por archivo
        })

    except Exception as e:
        print(f"[WARN] Error en {fp}: {e}")
        rows.append({
            "species": especie,
            "relpath": str(fp),
            "sr": np.nan,
            "T_frames_total": np.nan,
            "T_frames_valid": 0,
            "SC_hat_file_hz": np.nan,
            "error": f"{type(e).__name__}: {e}",
        })

df_sc_files = pd.DataFrame(rows)
print("Archivos procesados:", df_sc_files.shape[0])
df_sc_files.head()

In [ ]:
# Celda 4 — Resumen por especie y export de SC.csv

df_valid = df_sc_files.dropna(subset=["SC_hat_file_hz"]).copy()

df_species = (
    df_valid
    .groupby("species")
    .agg(
        n_muestras           = ("relpath", "count"),
        T_frames_valid_mean  = ("T_frames_valid", "mean"),
        SC_hat_mean_hz       = ("SC_hat_file_hz", "mean"),
        SC_hat_median_hz     = ("SC_hat_file_hz", "median"),
    )
    .reset_index()
    .sort_values("n_muestras", ascending=False)
)

print("Especies con datos:", df_species.shape[0])
df_species.head(10)

df_species.to_csv(OUT_CSV, index=False)
print(f"[OK] Archivo guardado en: {OUT_CSV}")
